Throughout the rest of the book, I use the following import conventions for NumPy and pandas:

In [1]:
import numpy as np
import pandas as pd

You may also find it easier to import Series and DataFrame into the local namespace since they are so frequently used:

In [2]:
from pandas import Series, DataFrame

# 1 Introduction to pandas

## 1.1 Series

A Series is a one-dimentional array-like object containing a sequence of values of the same type and an associated array of data labels, called its index.  

`pandas.Series` 就是带标签（index）的一维数组。可以理解成：
> Excel 里的一列（含行号和数据）

In [3]:
obj = pd.Series([4, 7, -5, 3])

obj

0    4
1    7
2   -5
3    3
dtype: int64

You can get the array representation and index object of the Series via its `array` and `index` attributes, respectively:

In [4]:
obj.array

<NumpyExtensionArray>
[4, 7, -5, 3]
Length: 4, dtype: int64

In [5]:
obj.index

RangeIndex(start=0, stop=4, step=1)

Often, you'll want to create a Series with an index identifying each data point with a label:

In [6]:
obj2 = pd.Series([4, 7, -5, 3], index=["d", "b", "a", "c"])

obj2

d    4
b    7
a   -5
c    3
dtype: int64

In [7]:
obj2.index

Index(['d', 'b', 'a', 'c'], dtype='object')

Compared with NumPy arrays, you can use labels in the index when selecting single values or a set of values:

In [8]:
obj2["a"]

np.int64(-5)

In [9]:
obj2["d"] = 6

obj2[["c", "a", "d"]]

c    3
a   -5
d    6
dtype: int64

Using NumPy functions or NumPy-like operations, such as filtering with a Boolean array, scalar multiplication, or applying math functions, will preserve the index-value link:

In [10]:
obj2[obj2 > 0]

d    6
b    7
c    3
dtype: int64

In [11]:
obj2 * 2

d    12
b    14
a   -10
c     6
dtype: int64

In [12]:
np.exp(obj2)

d     403.428793
b    1096.633158
a       0.006738
c      20.085537
dtype: float64

Another way to think about a Series is as a fixed-length, ordered dictionary, as it is a mapping of index values to data values. It can be used in many contexts where you might use a dictionary:

In [13]:
"b" in obj2

True

In [14]:
"e" in obj2

False

Should you have data contained in a Python dictionary, you can create a Series from it by passing the dictionary:

In [15]:
sdata = {"Ohio": 35000, "Texas": 71000, "Oregon": 16000, "Utah": 5000}

obj3 = pd.Series(sdata)

obj3

Ohio      35000
Texas     71000
Oregon    16000
Utah       5000
dtype: int64

A Series can be converted back to a dictionary with its to_dict method:

In [16]:
obj3.to_dict()

{'Ohio': 35000, 'Texas': 71000, 'Oregon': 16000, 'Utah': 5000}

You can override this by passing an index with the dictionary keys in the order you want them to appear in the resulting Series:

In [17]:
states = ["California", "Ohio", "Oregon", "Texas"]

obj4 = pd.Series(sdata, index=states)

obj4

California        NaN
Ohio          35000.0
Oregon        16000.0
Texas         71000.0
dtype: float64

> Here, three values found in sdata were placed in the appropriate locations, but since no value for "California" was found, it appears as NaN (Not a Number), which is considered in pandas to mark missing or NA values. Since "Utah" was not included in states, it is excluded from the resulting object.

I will use the terms “missing,” “NA,” or “null” interchangeably to refer to missing data. The `isna` and `notna` functions in pandas should be used to detect missing data:

In [18]:
pd.isna(obj4) # Alter: obj4.isna()

California     True
Ohio          False
Oregon        False
Texas         False
dtype: bool

In [19]:
pd.notna(obj4)

California    False
Ohio           True
Oregon         True
Texas          True
dtype: bool

A useful Series feature for many applications is that it automatically aligns by index label in arithmetic operations:

In [20]:
obj3 + obj4

California         NaN
Ohio           70000.0
Oregon         32000.0
Texas         142000.0
Utah               NaN
dtype: float64

Both the Series object itself and its index have a `name` attribute, which integrates with other areas of pandas functionality:

In [21]:
obj4.name = "population"

obj4.index.name = "state"

obj4

state
California        NaN
Ohio          35000.0
Oregon        16000.0
Texas         71000.0
Name: population, dtype: float64

A Series's index can be altered in place by assignment:

In [22]:
obj

0    4
1    7
2   -5
3    3
dtype: int64

In [23]:
obj.index = ["Bob", "Steve", "Jeff", "Ryan"]

obj

Bob      4
Steve    7
Jeff    -5
Ryan     3
dtype: int64

## 1.2 DataFrame

`pd.DataFrame` 是由多个 `Series` 按列组合而成的二维表格结构。
可以理解成：
> Excel 里的整个表格（含多列、多行）

There are many ways to construct a DataFrame, though one of the most common is from a dictionary of equal-length lists or NumPy arrays:

In [24]:
data = {
    "state": ["Ohio", "Ohio", "Ohio", "Nevada", "Nevada", "Nevada"],
    "year": [2000, 2001, 2002, 2001, 2002, 2003],
    "pop": [1.5, 1.7, 3.6, 2.4, 2.9, 3.2]
}

frame = pd.DataFrame(data)

frame

,state,year,pop
0,Ohio,2000,1.5
1,Ohio,2001,1.7
2,Ohio,2002,3.6
3,Nevada,2001,2.4
4,Nevada,2002,2.9
5,Nevada,2003,3.2


For large DataFrames, the `head` method selects only the **first five rows**:

In [25]:
frame.head()

,state,year,pop
0,Ohio,2000,1.5
1,Ohio,2001,1.7
2,Ohio,2002,3.6
3,Nevada,2001,2.4
4,Nevada,2002,2.9


Similarly, `tail` returns the last five rows:

In [26]:
frame.tail()

,state,year,pop
1,Ohio,2001,1.7
2,Ohio,2002,3.6
3,Nevada,2001,2.4
4,Nevada,2002,2.9
5,Nevada,2003,3.2


If you specify a sequence of columns, the DataFrame’s columns will be arranged in that order:

In [27]:
pd.DataFrame(data, columns=["year", "state", "pop"])

,year,state,pop
0,2000,Ohio,1.5
1,2001,Ohio,1.7
2,2002,Ohio,3.6
3,2001,Nevada,2.4
4,2002,Nevada,2.9
5,2003,Nevada,3.2


If you pass a column that isn’t contained in the dictionary, it will appear with missing values in the result:

In [28]:
frame2 = pd.DataFrame(data, columns=["year", "state", "pop", "debt"])

frame2

,year,state,pop,debt
0,2000,Ohio,1.5,NaN
1,2001,Ohio,1.7,NaN
2,2002,Ohio,3.6,NaN
3,2001,Nevada,2.4,NaN
4,2002,Nevada,2.9,NaN
5,2003,Nevada,3.2,NaN


A column in a DataFrame can be retrieved as a Series either by dictionary-like notation or by using the dot attribute notation:

In [29]:
frame2["state"]

0      Ohio
1      Ohio
2      Ohio
3    Nevada
4    Nevada
5    Nevada
Name: state, dtype: object

In [30]:
frame2.year

0    2000
1    2001
2    2002
3    2001
4    2002
5    2003
Name: year, dtype: int64

Rows can also be retrieved by position or name with the special `iloc` and `loc` attributes

In [31]:
frame2.loc[1]

year     2001
state    Ohio
pop       1.7
debt      NaN
Name: 1, dtype: object

In [32]:
frame2.iloc[2]

year     2002
state    Ohio
pop       3.6
debt      NaN
Name: 2, dtype: object

Columns can be modified by assignment. For example, the empty `debt` column could be assigned a scalar value or an array of values:

In [33]:
frame2["debt"] = 16.5

frame2

,year,state,pop,debt
0,2000,Ohio,1.5,16.5
1,2001,Ohio,1.7,16.5
2,2002,Ohio,3.6,16.5
3,2001,Nevada,2.4,16.5
4,2002,Nevada,2.9,16.5
5,2003,Nevada,3.2,16.5


In [34]:
frame2["debt"] = np.arange(6.0)

frame2

,year,state,pop,debt
0,2000,Ohio,1.5,0.0
1,2001,Ohio,1.7,1.0
2,2002,Ohio,3.6,2.0
3,2001,Nevada,2.4,3.0
4,2002,Nevada,2.9,4.0
5,2003,Nevada,3.2,5.0


When you are assigning lists or arrays to a column, the value’s length must match the length of the DataFrame. If you assign a Series, its labels will be realigned exactly to the DataFrame’s index, inserting missing values in any index values not present:

In [35]:
val = pd.Series([-1.2, -1.5, -1.7], index=[2, 4, 5])

frame2["debt"] = val

frame2

,year,state,pop,debt
0,2000,Ohio,1.5,NaN
1,2001,Ohio,1.7,NaN
2,2002,Ohio,3.6,-1.2
3,2001,Nevada,2.4,NaN
4,2002,Nevada,2.9,-1.5
5,2003,Nevada,3.2,-1.7


> Assigning a column that doesn’t exist will create a new column.

The `del` keyword will delete columns like with a dictionary. As an example, I first add a new column of Boolean values where the `state` column equals `"Ohio"`:

In [36]:
frame2["eastern"] = frame2["state"] == "Ohio"

frame2

,year,state,pop,debt,eastern
0,2000,Ohio,1.5,NaN,True
1,2001,Ohio,1.7,NaN,True
2,2002,Ohio,3.6,-1.2,True
3,2001,Nevada,2.4,NaN,False
4,2002,Nevada,2.9,-1.5,False
5,2003,Nevada,3.2,-1.7,False


The `del` method can then be used to remove this column:

In [37]:
del frame2["eastern"]

frame2.columns

Index(['year', 'state', 'pop', 'debt'], dtype='object')

Another common form of data is a nested dictionary of dictionaries:

In [38]:
populations = {
    "Ohio": {2000: 1.5, 2001: 1.7, 2002: 3.6},
    "Nevada": {2001: 2.4, 2002: 2.9}
}

frame3 = pd.DataFrame(populations)

frame3

,Ohio,Nevada
2000,1.5,NaN
2001,1.7,2.4
2002,3.6,2.9


You can transpose the DataFrame (swap rows and columns) with similar syntax to a NumPy array:

In [39]:
frame3.T

,2000,2001,2002
Ohio,1.5,1.7,3.6
Nevada,NaN,2.4,2.9


The keys in the inner dictionaries are combined to form the index in the result. This isn’t true if an explicit index is specified:

In [40]:
pd.DataFrame(populations, index=[2001, 2002, 2003])

,Ohio,Nevada
2001,1.7,2.4
2002,3.6,2.9
2003,NaN,NaN


Dictionaries of Series are treated in much the same way:

In [41]:
pdata = {
    "Ohio": frame3["Ohio"][:-1],
    "Nevada": frame3["Nevada"][:2]
}

pd.DataFrame(pdata)

,Ohio,Nevada
2000,1.5,NaN
2001,1.7,2.4


If a DataFrame’s index and columns have their name attributes set, these will also be displayed:

In [42]:
frame3

,Ohio,Nevada
2000,1.5,NaN
2001,1.7,2.4
2002,3.6,2.9


In [43]:
frame3.index.name = "year"
frame3.columns.name = "state"

frame3

state,Ohio,Nevada
year,,
2000,1.5,NaN
2001,1.7,2.4
2002,3.6,2.9


Unlike Series, DataFrame does not have a `name` attribute. DataFrame's `to_numpy` method returns the data contained in the DataFrame as a two-dimensional ndarray:

In [44]:
frame3.to_numpy()

array([[1.5, nan],
       [1.7, 2.4],
       [3.6, 2.9]])

If the DataFrame’s columns are different data types, the data type of the returned array will be chosen to accommodate all of the columns:

In [45]:
frame2.to_numpy()

array([[2000, 'Ohio', 1.5, nan],
       [2001, 'Ohio', 1.7, nan],
       [2002, 'Ohio', 3.6, -1.2],
       [2001, 'Nevada', 2.4, nan],
       [2002, 'Nevada', 2.9, -1.5],
       [2003, 'Nevada', 3.2, -1.7]], dtype=object)

## 1.3 Index Objects

pandas’s Index objects are responsible for holding the axis labels (including a DataFrame's column names) and other metadata (like the axis name or names). Any array or other sequence of labels you use when constructing a Series or DataFrame is internally converted to an Index:

In [46]:
obj = pd.Series(np.arange(3), index=["a", "b", "c"])

index = obj.index

index

Index(['a', 'b', 'c'], dtype='object')

In [47]:
index[1:]

Index(['b', 'c'], dtype='object')

Index objects are immutable and thus can’t be modified by the user:  
```python
index[1] = "d" # TypeError
```

Immutability makes it safer to share Index objects among data structures:

In [48]:
labels = pd.Index(np.arange(3))

labels

Index([0, 1, 2], dtype='int64')

In [49]:
obj2 = pd.Series([1.5, -2.5, 0], index = labels)

obj2

0    1.5
1   -2.5
2    0.0
dtype: float64

In [50]:
obj2.index is labels

True

In addition to being array-like, an Index also behaves like a fixed-size set:

In [51]:
frame3

state,Ohio,Nevada
year,,
2000,1.5,NaN
2001,1.7,2.4
2002,3.6,2.9


In [52]:
frame3.columns

Index(['Ohio', 'Nevada'], dtype='object', name='state')

In [53]:
"Ohio" in frame3.columns

True

In [54]:
2003 in frame3.index

False

Unlike Python sets, a pandas Index can contain duplicate labels:

In [55]:
pd.Index(["foo", "foo", "bar", "bar"])

Index(['foo', 'foo', 'bar', 'bar'], dtype='object')

# 2 Essential Functionality

This section will walk you through the fundamental mechanics of interacting with the data contained in a Series or DataFrame. In the chapters to come, we will delve more deeply into data analysis and manipulation topics using pandas. 

## 2.1 Reindexing

An important method on pandas objects is reindex, which means to create a new object with the values rearranged to align with the new index.

In [56]:
obj = pd.Series([4.5, 7.2, -5.3, 3.6], index=["d", "b", "a", "c"])

obj

d    4.5
b    7.2
a   -5.3
c    3.6
dtype: float64

Calling `reindex` on this Series rearranges the data according to the new index, introducing missing values if any index values were not already present:

In [57]:
obj2 = obj.reindex(["a", "b", "c", "d", "e"])

obj2

a   -5.3
b    7.2
c    3.6
d    4.5
e    NaN
dtype: float64

For ordered data like time series, you may want to do some interpolation or filling of values when reindexing. The `method` option allows us to do this, using a method such as `ffill`, which forward-fills the values:

In [58]:
obj3 = pd.Series(["blue", "purple", "yellow"], index=[0, 2, 4])

obj3

0      blue
2    purple
4    yellow
dtype: object

In [59]:
obj3.reindex(np.arange(6), method="ffill")

0      blue
1      blue
2    purple
3    purple
4    yellow
5    yellow
dtype: object

With DataFrame, reindex can alter the (row) index, columns, or both. When passed only a sequence, it reindexes the rows in the result:

In [60]:
frame = pd.DataFrame(np.arange(9).reshape((3, 3)),
                    index=["a", "c", "d"],
                    columns=["Ohio", "Texas", "California"])

frame

,Ohio,Texas,California
a,0,1,2
c,3,4,5
d,6,7,8


In [61]:
frame2 = frame.reindex(index=["a", "b", "c", "d"])

frame2

,Ohio,Texas,California
a,0.0,1.0,2.0
b,NaN,NaN,NaN
c,3.0,4.0,5.0
d,6.0,7.0,8.0


The columns can be reindexed with the `columns` keyword:

In [62]:
states = ["Texas", "Utah", "California"]

frame.reindex(columns=states)

,Texas,Utah,California
a,1,NaN,2
c,4,NaN,5
d,7,NaN,8


> Because `"Ohio"` was not in `states`, the data for that column is dropped from the result.

Another way to reindex a particular axis is to pass the new axis labels as a positional argument and then specify the axis to reindex with the `axis`keyword:

In [63]:
frame.reindex(states, axis="columns")

,Texas,Utah,California
a,1,NaN,2
c,4,NaN,5
d,7,NaN,8


As we'll explore later in Selection on DataFrame with loc and iloc, you can also reindex by using the `loc` operator, and many users prefer to always do it this way. This works only if all of the new index labels already exist in the DataFrame (whereas `reindex` will insert missing data for new labels):

In [64]:
frame.loc[["a", "d", "c"], ["California", "Texas"]]

,California,Texas
a,2,1
d,8,7
c,5,4


## 2.2 Dropping Entries from an Axis

Dropping one or more entries from an axis is simple if you already have an index array or list without those entries, since you can use the `reindex` method or `.loc`-based indexing. As that can require a bit of munging and set logic, the `drop` method will return a new object with the indicated value or values deleted from an axis:

In [65]:
obj = pd.Series(np.arange(5.), index=["a", "b", "c", "d", "e"])

obj

a    0.0
b    1.0
c    2.0
d    3.0
e    4.0
dtype: float64

In [66]:
new_obj = obj.drop("c")

new_obj

a    0.0
b    1.0
d    3.0
e    4.0
dtype: float64

In [67]:
obj.drop(["d", "c"])

a    0.0
b    1.0
e    4.0
dtype: float64

With DataFrame, index values can be deleted from either axis. To illustrate this, we first create an example DataFrame:

In [68]:
data = pd.DataFrame(np.arange(16).reshape((4, 4)),
                     index=["Ohio", "Colorado", "Utah", "New York"],
                     columns=["one", "two", "three", "four"])
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
Utah,8,9,10,11
New York,12,13,14,15


Calling `drop` with a sequence of labels will drop values from the row labels (axis 0):

In [69]:
data.drop(index=["Colorado", "Ohio"])

,one,two,three,four
Utah,8,9,10,11
New York,12,13,14,15


To drop labels from the columns, instead use the `columns` keyword:

In [70]:
data.drop(columns=["two"]) # data.drop("two", axis=1)

,one,three,four
Ohio,0,2,3
Colorado,4,6,7
Utah,8,10,11
New York,12,14,15


In [71]:
data.drop(["two", "four"], axis="columns")

,one,three
Ohio,0,2
Colorado,4,6
Utah,8,10
New York,12,14


## 2.3 Indexing, Selection and Filtering

Series indexing (`obj[...]`) works analogously to NumPy array indexing, except you can use the Series’s index values instead of only integers. Here are some examples of this:

In [72]:
obj = pd.Series(np.arange(4.), index=["a", "b", "c", "d"])

obj

a    0.0
b    1.0
c    2.0
d    3.0
dtype: float64

In [73]:
obj["b"]

np.float64(1.0)

In [74]:
obj[2:4]

c    2.0
d    3.0
dtype: float64

In [75]:
obj[["b", "a", "d"]]

b    1.0
a    0.0
d    3.0
dtype: float64

In [76]:
obj[obj < 2]

a    0.0
b    1.0
dtype: float64

In [77]:
obj.loc[["b", "a", "d"]]

b    1.0
a    0.0
d    3.0
dtype: float64

The reason to prefer `loc` is because of the different treatment of integers when indexing with `[]`. Regular `[]`-based indexing will treat integers as labels if the index contains integers, so the behavior differs depending on the data type of the index. For example:

In [78]:
obj1 = pd.Series([1, 2, 3], index=[2, 0, 1])
obj2 = pd.Series([1, 2, 3], index=["a", "b", "c"])

obj1

2    1
0    2
1    3
dtype: int64

In [79]:
obj2

a    1
b    2
c    3
dtype: int64

In [80]:
obj1[[0, 1, 2]]

0    2
1    3
2    1
dtype: int64

When using `loc`, the expression `obj.loc[[0, 1, 2]]` will fail when the index does not contain integers:
```python
obj2.loc[[0, 1]] # KeyError
```

Since `loc` operator indexes exclusively with labels, there is also an `iloc` operator that indexes exclusively with integers to work consistently whether or not the index contains integers:

In [81]:
obj1.iloc[[0, 1, 2]]

2    1
0    2
1    3
dtype: int64

In [82]:
obj2.iloc[[0, 1, 2]]

a    1
b    2
c    3
dtype: int64

Assigning values using these methods modifies the corresponding section of the Series:


In [83]:
obj2.loc["b":"c"] = 5

obj2

a    1
b    5
c    5
dtype: int64

> the endpoint is inclusive

Indexing into a DataFrame retrieves one or more columns either with a single value or sequence:

In [84]:
data = pd.DataFrame(np.arange(16).reshape((4, 4)),
                     index=["Ohio", "Colorado", "Utah", "New York"],
                     columns=["one", "two", "three", "four"])
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
Utah,8,9,10,11
New York,12,13,14,15


In [85]:
data["two"]

Ohio         1
Colorado     5
Utah         9
New York    13
Name: two, dtype: int64

In [86]:
data[["three", "one"]]

,three,one
Ohio,2,0
Colorado,6,4
Utah,10,8
New York,14,12


Indexing like this has a few special cases. The first is slicing or selecting data with a Boolean array:

In [87]:
data[:2]

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7


In [88]:
data[data["three"] > 5]

,one,two,three,four
Colorado,4,5,6,7
Utah,8,9,10,11
New York,12,13,14,15


> The row selection syntax `data[:2]` is provided as a convenience. Passing a single element or a list to the `[]` operator selects columns.

Another use case is indexing with a Boolean DataFrame, such as one produced by a scalar comparison. Consider a DataFrame with all Boolean values produced by comparing with a scalar value:

In [89]:
data < 5

,one,two,three,four
Ohio,True,True,True,True
Colorado,True,False,False,False
Utah,False,False,False,False
New York,False,False,False,False


We can use this DataFrame to assign the value 0 to each location with the value `True`, like so:

In [90]:
data[data < 5] = 0

data

,one,two,three,four
Ohio,0,0,0,0
Colorado,0,5,6,7
Utah,8,9,10,11
New York,12,13,14,15


## 2.4 Selection on DataFrame with loc can iloc

Like Series, DataFrame has special attributes `loc` and `iloc` for label-based and integer-based indexing, respectively. Since DataFrame is two-dimensional, you can select a subset of the rows and columns with NumPy-like notation using either axis labels (`loc`) or integers (`iloc`).  

As a first example, let's select a single row by label:

In [91]:
data

,one,two,three,four
Ohio,0,0,0,0
Colorado,0,5,6,7
Utah,8,9,10,11
New York,12,13,14,15


In [92]:
data.loc["Colorado"]

one      0
two      5
three    6
four     7
Name: Colorado, dtype: int64

To select multiple roles, creating a new DataFrame, pass a sequence of labels:

In [93]:
data.loc[["Colorado", "New York"]]

,one,two,three,four
Colorado,0,5,6,7
New York,12,13,14,15


You can combine both row and column selection in `loc` by separating the selections with a comma:

In [94]:
data.loc["Colorado", ["two", "three"]]

two      5
three    6
Name: Colorado, dtype: int64

We'll then perform some similar selections with integers using `iloc`:

In [95]:
data.iloc[2]

one       8
two       9
three    10
four     11
Name: Utah, dtype: int64

In [96]:
data.iloc[[1, 2]]

,one,two,three,four
Colorado,0,5,6,7
Utah,8,9,10,11


In [97]:
data.iloc[2, [3, 0, 1]]

four    11
one      8
two      9
Name: Utah, dtype: int64

In [98]:
data.iloc[[1, 2], [3, 0, 1]]

,four,one,two
Colorado,7,0,5
Utah,11,8,9


Both indexing functions work with slices in addition to single labels or lists of labels:

In [99]:
data.loc[:"Utah", "two":]

,two,three,four
Ohio,0,0,0
Colorado,5,6,7
Utah,9,10,11


In [100]:
data.iloc[:, :3][data.three > 7]

,one,two,three
Utah,8,9,10
New York,12,13,14


Boolean arrays can be used with `loc` but not `iloc`:

In [101]:
data.loc[data.three >= 2]

,one,two,three,four
Colorado,0,5,6,7
Utah,8,9,10,11
New York,12,13,14,15


## 2.5 Integer indexing pitfalls

In [102]:
ser = pd.Series(np.arange(3.))

ser

0    0.0
1    1.0
2    2.0
dtype: float64

```python
ser[-1] # ValueError
```
In this case, pandas could “fall back” on integer indexing, but it is difficult to do this in general without introducing subtle bugs into the user code. Here we have an index containing `0`, `1`, and `2`, but pandas does not want to guess what the user wants (label-based indexing or position-based):

If you have an axis index containing integers, data selection will always be label oriented. As I said above, if you use `loc` (for labels) or `iloc` (for integers) you will get exactly what you want:

In [103]:
ser.iloc[-1]

np.float64(2.0)

On the other hand, slicing with integers is always integer oriented:

In [104]:
ser[:2]

0    0.0
1    1.0
dtype: float64

As a result of these pitfalls, it is best to always prefer indexing with loc and iloc to avoid ambiguity.

## 2.6 Pitfalls with chained indexing

In the previous section we looked at how you can do flexible selections on a DataFrame using `loc` and `iloc`. These indexing attributes can also be used to modify DataFrame objects in place, but doing so requires some care.  

For example, in the example DataFrame above, we can assign to a column or row by label or integer position:

In [105]:
data.loc[:, "one"] = 1

data

,one,two,three,four
Ohio,1,0,0,0
Colorado,1,5,6,7
Utah,1,9,10,11
New York,1,13,14,15


In [106]:
data.iloc[2] = 5

data

,one,two,three,four
Ohio,1,0,0,0
Colorado,1,5,6,7
Utah,5,5,5,5
New York,1,13,14,15


In [107]:
data.loc[data["four"] > 5] = 3

data

,one,two,three,four
Ohio,1,0,0,0
Colorado,3,3,3,3
Utah,5,5,5,5
New York,3,3,3,3


In [108]:
data.loc[data.three == 5, "three"] = 6

data

,one,two,three,four
Ohio,1,0,0,0
Colorado,3,3,3,3
Utah,5,5,6,5
New York,3,3,3,3


A good rule of thumb is to avoid chained indexing when doing assignments.

## 2.7 Arithmetic and Data Alignment

pandas can make it much simpler to work with objects that have different indexes. For example, when you add objects, if any index pairs are not the same, the respective index in the result will be the union of the index pairs. Let’s look at an example:

In [109]:
s1 = pd.Series([7.3, 02.5, 3.4, 1.5], index=["a", "c", "d", "e"])

s2 = pd.Series([-2.1, 3.6, -1.5, 4, 3.1], index=["a", "c", "e", "f", "g"])

s1

a    7.3
c    2.5
d    3.4
e    1.5
dtype: float64

In [110]:
s2

a   -2.1
c    3.6
e   -1.5
f    4.0
g    3.1
dtype: float64

Adding these yields:

In [111]:
s1 + s2

a    5.2
c    6.1
d    NaN
e    0.0
f    NaN
g    NaN
dtype: float64

The internal data alignment introduces missing values in the label locations that don’t overlap. Missing values will then propagate in further arithmetic computations.

In the case of DataFrame, alignment is performed on both rows and columns:

In [112]:
df1 = pd.DataFrame(np.arange(9.).reshape((3, 3)), columns=list("bcd"),
                  index=["Ohio", "Texas", "Colorado"])

df2 = pd.DataFrame(np.arange(12.).reshape((4, 3)), columns=list("bde"),
                  index=["Utah", "Ohio", "Texas", "Oregon"])

df1

,b,c,d
Ohio,0.0,1.0,2.0
Texas,3.0,4.0,5.0
Colorado,6.0,7.0,8.0


In [113]:
df2

,b,d,e
Utah,0.0,1.0,2.0
Ohio,3.0,4.0,5.0
Texas,6.0,7.0,8.0
Oregon,9.0,10.0,11.0


Adding these returns a DataFrame with index and columns that are the unions of the ones in each DataFrame:

In [114]:
df1 + df2

,b,c,d,e
Colorado,NaN,NaN,NaN,NaN
Ohio,3.0,NaN,6.0,NaN
Oregon,NaN,NaN,NaN,NaN
Texas,9.0,NaN,12.0,NaN
Utah,NaN,NaN,NaN,NaN


Since the "c" and "e" columns are not found in both DataFrame objects, they appear as missing in the result. The same holds for the rows with labels that are not common to both objects.

If you add DataFrame objects with no column or row labels in common, the result will contain all nulls:

In [115]:
df1 = pd.DataFrame({"A": [1, 2]})

df2 = pd.DataFrame({"B": [3, 4]})

df1

,A
0,1
1,2


In [116]:
df2

,B
0,3
1,4


In [117]:
df1 + df2

,A,B
0,NaN,NaN
1,NaN,NaN


## 2.8 Arithmetic methods with fill values

In arithmetic operations between differently indexed objects, you might want to fill with a special value, like 0, when an axis label is found in one object but not the other. Here is an example where we set a particular value to NA (null) by assigning `np.nan` to it:

In [118]:
df1 = pd.DataFrame(np.arange(12.).reshape((3, 4)), columns=list("abcd"))

df2 = pd.DataFrame(np.arange(20.).reshape((4, 5)), columns=list("abcde"))

df1

,a,b,c,d
0,0.0,1.0,2.0,3.0
1,4.0,5.0,6.0,7.0
2,8.0,9.0,10.0,11.0


In [119]:
df2.loc[1, "b"] = np.nan

df2

,a,b,c,d,e
0,0.0,1.0,2.0,3.0,4.0
1,5.0,NaN,7.0,8.0,9.0
2,10.0,11.0,12.0,13.0,14.0
3,15.0,16.0,17.0,18.0,19.0


Adding these results in missing values in the locations that don’t overlap:

In [120]:
df1 + df2

,a,b,c,d,e
0,0.0,2.0,4.0,6.0,NaN
1,9.0,NaN,13.0,15.0,NaN
2,18.0,20.0,22.0,24.0,NaN
3,NaN,NaN,NaN,NaN,NaN


Using the `add` method on `df1`, I pass `df2` and an argument to `fill_value`, which substitutes the passed value for any missing values in the operation:

In [121]:
df1.add(df2, fill_value=0)

,a,b,c,d,e
0,0.0,2.0,4.0,6.0,4.0
1,9.0,5.0,13.0,15.0,9.0
2,18.0,20.0,22.0,24.0,14.0
3,15.0,16.0,17.0,18.0,19.0


Relatedly, when reindexing a Series or DataFrame, you can also specify a different fill value:

In [122]:
df1.reindex(columns=df2.columns, fill_value=0)

,a,b,c,d,e
0,0.0,1.0,2.0,3.0,0
1,4.0,5.0,6.0,7.0,0
2,8.0,9.0,10.0,11.0,0


## 2.9 Operations between DataFrame and Series

As with NumPy arrays of different dimensions, arithmetic between DataFrame and Series is also defined. First, as a motivating example, consider the difference between a two-dimensional array and one of its rows:

In [123]:
arr = np.arange(12.).reshape((3, 4))

arr

array([[ 0.,  1.,  2.,  3.],
       [ 4.,  5.,  6.,  7.],
       [ 8.,  9., 10., 11.]])

In [124]:
arr[0]

array([0., 1., 2., 3.])

In [125]:
arr - arr[0]

array([[0., 0., 0., 0.],
       [4., 4., 4., 4.],
       [8., 8., 8., 8.]])

When we subtract `arr[0]` from arr, the subtraction is performed once for each row. This is referred to as **broadcasting**. Operations between a DataFrame and a Series are similar:

In [126]:
frame = pd.DataFrame(np.arange(12.).reshape((4, 3)),
                      columns=list("bde"),
                      index=["Utah", "Ohio", "Texas", "Oregon"])

series = frame.iloc[0]

frame

,b,d,e
Utah,0.0,1.0,2.0
Ohio,3.0,4.0,5.0
Texas,6.0,7.0,8.0
Oregon,9.0,10.0,11.0


In [127]:
series

b    0.0
d    1.0
e    2.0
Name: Utah, dtype: float64

By default, arithmetic between DataFrame and Series matches the index of the Series on the columns of the DataFrame, broadcasting down the rows:

In [128]:
frame - series

,b,d,e
Utah,0.0,0.0,0.0
Ohio,3.0,3.0,3.0
Texas,6.0,6.0,6.0
Oregon,9.0,9.0,9.0


If an index value is not found in either the DataFrame’s columns or the Series’s index, the objects will be reindexed to form the union:

In [129]:
series2 = pd.Series(np.arange(3), index=["b", "e", "f"])

series2

b    0
e    1
f    2
dtype: int64

In [130]:
frame

,b,d,e
Utah,0.0,1.0,2.0
Ohio,3.0,4.0,5.0
Texas,6.0,7.0,8.0
Oregon,9.0,10.0,11.0


In [131]:
frame + series2

,b,d,e,f
Utah,0.0,NaN,3.0,NaN
Ohio,3.0,NaN,6.0,NaN
Texas,6.0,NaN,9.0,NaN
Oregon,9.0,NaN,12.0,NaN


If you want to instead broadcast over the columns, matching on the rows, you have to use one of the arithmetic methods and specify to match over the index. For example:

In [132]:
series3 = frame["d"]

series3

Utah       1.0
Ohio       4.0
Texas      7.0
Oregon    10.0
Name: d, dtype: float64

In [133]:
frame.sub(series3, axis="index")

,b,d,e
Utah,-1.0,0.0,1.0
Ohio,-1.0,0.0,1.0
Texas,-1.0,0.0,1.0
Oregon,-1.0,0.0,1.0


The axis that you pass is the axis to match on. In this case we mean to match on the DataFrame’s row index (`axis="index"`) and broadcast across the columns.

## 2.10 Function Application and Mapping  

NumPy ufuncs (element-wise array methods) also work with pandas objects:

In [134]:
frame = pd.DataFrame(np.random.standard_normal((4, 3)),
                   columns=list("bde"),
                   index=["Utah", "Ohio", "Texas", "Oregon"])

frame

,b,d,e
Utah,-0.381617,0.769599,-0.343761
Ohio,0.694011,-1.126493,-0.545340
Texas,-0.119417,-0.196361,0.706610
Oregon,1.236608,-0.301553,1.605393


In [135]:
np.abs(frame)

,b,d,e
Utah,0.381617,0.769599,0.343761
Ohio,0.694011,1.126493,0.545340
Texas,0.119417,0.196361,0.706610
Oregon,1.236608,0.301553,1.605393


Another frequent operation is applying a function on one-dimensional arrays to each column or row. DataFrame’s `apply` method does exactly this:

In [136]:
def f1(x):
    return x.max() - x.min()

frame.apply(f1)

b    1.618224
d    1.896092
e    2.150734
dtype: float64

If you pass `axis="columns"` to `apply`, the function will be invoked once per row instead. A helpful way to think about this is as "apply across the columns":

In [137]:
frame.apply(f1, axis="columns")

Utah      1.151216
Ohio      1.820504
Texas     0.902971
Oregon    1.906947
dtype: float64

The function passed to `apply` need not return a scalar value; it can also return a Series with multiple values:

In [138]:
def f2(x):
    return pd.Series([x.min(), x.max()], index=["min", "max"])

frame.apply(f2)

,b,d,e
min,-0.381617,-1.126493,-0.545340
max,1.236608,0.769599,1.605393


Element-wise Python functions can be used, too. Suppose you wanted to compute a formatted string from each floating-point value in `frame`. You can do this with `applymap`:

In [139]:
def my_format(x):
    return f"{x:.2f}"

frame.map(my_format)

,b,d,e
Utah,-0.38,0.77,-0.34
Ohio,0.69,-1.13,-0.55
Texas,-0.12,-0.20,0.71
Oregon,1.24,-0.30,1.61


In [140]:
frame["e"].map(my_format)

Utah      -0.34
Ohio      -0.55
Texas      0.71
Oregon     1.61
Name: e, dtype: object

## 2.11 Sorting and Ranking

Sorting a dataset by some criterion is another important built-in operation. To sort lexicographically by row or column label, use the `sort_index` method, which returns a new, sorted object:

In [141]:
obj = pd.Series(np.arange(4), index=["d", "a", "b", "c"])

obj

d    0
a    1
b    2
c    3
dtype: int64

In [142]:
obj.sort_index()

a    1
b    2
c    3
d    0
dtype: int64

With a DataFrame, you can sort by index on either axis:

In [143]:
frame = pd.DataFrame(np.arange(8).reshape((2, 4)),
                     index=["three", "one"],
                     columns=["d", "a", "b", "c"])

frame

,d,a,b,c
three,0,1,2,3
one,4,5,6,7


In [144]:
frame.sort_index()

,d,a,b,c
one,4,5,6,7
three,0,1,2,3


In [145]:
frame.sort_index(axis="columns")

,a,b,c,d
three,1,2,3,0
one,5,6,7,4


The data is sorted in ascending order by default but can be sorted in descending order, too:

In [146]:
frame.sort_index(axis="columns", ascending=False)

,d,c,b,a
three,0,3,2,1
one,4,7,6,5


To sort a Series by its values, use its `sort_values` method:

In [147]:
obj = pd.Series([4, 7, -3, 2])

obj.sort_values()

2   -3
3    2
0    4
1    7
dtype: int64

Any missing values are sorted to the end of the Series by default:

In [148]:
obj = pd.Series([4, np.nan, 7, np.nan, -3, 2])

obj.sort_values()

4   -3.0
5    2.0
0    4.0
2    7.0
1    NaN
3    NaN
dtype: float64

Missing values can be sorted to the start instead by using the `na_position` option:

In [149]:
obj.sort_values(na_position="first")

1    NaN
3    NaN
4   -3.0
5    2.0
0    4.0
2    7.0
dtype: float64

When sorting a DataFrame, you can use the data in one or more columns as the sort keys. To do so, pass one or more column names to `sort_values`:

In [150]:
frame = pd.DataFrame({"b": [4, 7, -3, 2], "a": [0, 1, 0, 1]})

frame

,b,a
0,4,0
1,7,1
2,-3,0
3,2,1


In [151]:
frame.sort_values("b")

,b,a
2,-3,0
3,2,1
0,4,0
1,7,1


To sort by multiple columns, pass a list of names:

In [152]:
frame.sort_values(["a", "b"])

,b,a
2,-3,0
0,4,0
3,2,1
1,7,1


Ranking assigns ranks from one through the number of valid data points in an array, starting from the lowest value. The `rank` methods for Series and DataFrame are the place to look; by default, `rank` breaks ties by assigning each group the mean rank:  

值 4 出现两次（index 3 和 6）  
它们本来应该是：
* 排名 4
* 排名 5  
但 重复值用平均排名 → (4 + 5) / 2 = 4.5  
所以 index 3 和 index 6 都为 4.5

值 7 也出现两次（index 0 和 2）
它们本来应该是：
* 排名 6
* 排名 7  
平均排名 = (6 + 7) / 2 = 6.5  
所以 index 0 和 index 2 都是 6.5

In [153]:
obj = pd.Series([7, -5, 7, 4, 2, 0, 4])

obj.rank()

0    6.5
1    1.0
2    6.5
3    4.5
4    3.0
5    2.0
6    4.5
dtype: float64

Ranks can also be assigned according to the order in which they’re observed in the data:

In [154]:
obj.rank(method="first")

0    6.0
1    1.0
2    7.0
3    4.0
4    3.0
5    2.0
6    5.0
dtype: float64

You can rank in descending order, too:

In [155]:
obj.rank(ascending=False)

0    1.5
1    7.0
2    1.5
3    3.5
4    5.0
5    6.0
6    3.5
dtype: float64

DataFrame can compute ranks over the rows or the columns:

In [156]:
frame = pd.DataFrame({"b": [4.3, 7, -3, 2], "a": [0, 1, 0, 1],
                       "c": [-2, 5, 8, -2.5]})

frame

,b,a,c
0,4.3,0,-2.0
1,7.0,1,5.0
2,-3.0,0,8.0
3,2.0,1,-2.5


In [158]:
frame.rank(axis="columns")

,b,a,c
0,3.0,2.0,1.0
1,3.0,1.0,2.0
2,1.0,2.0,3.0
3,3.0,2.0,1.0


## 2.12 Axis Indexes with Duplicate Labels

Up until now almost all of the examples we have looked at have unique axis labels (index values). While many pandas functions (like `reindex`) require that the labels be unique, it’s not mandatory. Let’s consider a small Series with duplicate indices:

In [159]:
obj = pd.Series(np.arange(5), index=["a", "a", "b", "b", "c"])

obj

a    0
a    1
b    2
b    3
c    4
dtype: int64

The `is_unique` property of the index can tell you whether or not its labels are unique:

In [160]:
obj.index.is_unique

False

Data selection is one of the main things that behaves differently with duplicates. Indexing a label with multiple entries returns a Series, while single entries return a scalar value:

In [161]:
obj["a"]

a    0
a    1
dtype: int64

In [162]:
obj["c"]

np.int64(4)

This can make your code more complicated, as the output type from indexing can vary based on whether or not a label is repeated.

The same logic extends to indexing rows (or columns) in a DataFrame:

In [163]:
df = pd.DataFrame(np.random.standard_normal((5, 3)), index=["a", "a", "b", "b", "c"])

df

,0,1,2
a,-0.198218,-0.525956,0.911628
a,0.889006,-0.490863,1.779644
b,0.537843,0.206153,-1.027223
b,1.341933,0.192045,-0.336753
c,0.242630,-0.157332,1.464273


In [164]:
df.loc["b"]

,0,1,2
b,0.537843,0.206153,-1.027223
b,1.341933,0.192045,-0.336753


In [165]:
df.loc["c"]

0    0.242630
1   -0.157332
2    1.464273
Name: c, dtype: float64

# 3 Summarizing and Computing Descriptive Statistics

In [166]:
df = pd.DataFrame([[1.4, np.nan], [7.1, -4.5],
                    [np.nan, np.nan], [0.75, -1.3]],
                   index=["a", "b", "c", "d"],
                   columns=["one", "two"])

df

,one,two
a,1.40,NaN
b,7.10,-4.5
c,NaN,NaN
d,0.75,-1.3


Calling DataFrame’s `sum` method returns a Series containing column sums:

In [167]:
df.sum()

one    9.25
two   -5.80
dtype: float64

Passing `axis="columns"` or `axis=1` sums across the columns instead:

In [169]:
df.sum(axis="columns")

a    1.40
b    2.60
c    0.00
d   -0.55
dtype: float64

When an entire row or column contains all NA values, the sum is 0, whereas if any value is not NA, then the result is NA. This can be disabled with the `skipna` option, in which case any NA value in a row or column names the corresponding result NA:

In [170]:
df.sum(axis="index", skipna=False)

one   NaN
two   NaN
dtype: float64

In [173]:
df.sum(axis="columns", skipna=False)

a     NaN
b    2.60
c     NaN
d   -0.55
dtype: float64

Some aggregations, like `mean`, require at least one non-NA value to yield a value result, so here we have:

In [174]:
df.mean(axis="columns")

a    1.400
b    1.300
c      NaN
d   -0.275
dtype: float64

Some methods, like `idxmin` and `idxmax`, return indirect statistics, like the index value where the minimum or maximum values are attained:

In [175]:
df.idxmax()

one    b
two    d
dtype: object

Other methods are accumulations:

In [176]:
df.cumsum()

,one,two
a,1.40,NaN
b,8.50,-4.5
c,NaN,NaN
d,9.25,-5.8


Some methods are neither reductions nor accumulations. `describe` is one such example, producing multiple summary statistics in one shot:

In [177]:
df.describe()

,one,two
count,3.000000,2.000000
mean,3.083333,-2.900000
std,3.493685,2.262742
min,0.750000,-4.500000
25%,1.075000,-3.700000
50%,1.400000,-2.900000
75%,4.250000,-2.100000
max,7.100000,-1.300000


On nonnumeric data, `describe` produces alternative summary statistics:

In [179]:
obj = pd.Series(["a", "a", "b", "c"] * 4)

obj.describe()

count     16
unique     3
top        a
freq       8
dtype: object

## 3.1 Unique Values, Value Counts, and Membership

Another class of related methods extracts information about the values contained in a one-dimensional Series. To illustrate these, consider this example:

In [180]:
obj = pd.Series(["c", "a", "d", "a", "a", "b", "b", "c", "c"])

The first function is `unique`, which gives you an array of the unique values in a Series:

In [182]:
uniques = obj.unique()

uniques

array(['c', 'a', 'd', 'b'], dtype=object)

The unique values are not necessarily returned in the order in which they first appear, and not in sorted order, but they could be sorted after the fact if needed (`uniques.sort()`). Relatedly, `value_counts` computes a Series containing value frequencies:

In [183]:
obj.value_counts()

c    3
a    3
b    2
d    1
Name: count, dtype: int64

The Series is sorted by value in descending order as a convenience.

`isin` performs a vectorized set membership check and can be useful in filtering a dataset down to a subset of values in a Series or column in a DataFrame:

In [184]:
obj

0    c
1    a
2    d
3    a
4    a
5    b
6    b
7    c
8    c
dtype: object

In [187]:
mask = obj.isin(["b", "c"])

mask

0     True
1    False
2    False
3    False
4    False
5     True
6     True
7     True
8     True
dtype: bool

In [188]:
obj[mask]

0    c
5    b
6    b
7    c
8    c
dtype: object

In some cases, you may want to compute a histogram on multiple related columns in a DataFrame. Here’s an example:

In [189]:
data = pd.DataFrame({"Qu1": [1, 3, 4, 3, 4],
                      "Qu2": [2, 3, 1, 2, 3],
                      "Qu3": [1, 5, 2, 4, 4]})

data

,Qu1,Qu2,Qu3
0,1,2,1
1,3,3,5
2,4,1,2
3,3,2,4
4,4,3,4


We can compute the value counts for a single column, like so:

In [191]:
data["Qu1"].value_counts().sort_index()

Qu1
1    1
3    2
4    2
Name: count, dtype: int64

To compute this for all columns, pass `pandas.value_counts` to the DataFrame’s `apply` method:

In [197]:
result = data.apply(pd.value_counts).fillna(0)

result

C:\Users\user\AppData\Local\Temp\ipykernel_9628\3698396091.py:1: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  result = data.apply(pd.value_counts).fillna(0)


,Qu1,Qu2,Qu3
1,1.0,1.0,1.0
2,0.0,2.0,1.0
3,2.0,2.0,0.0
4,2.0,0.0,2.0
5,0.0,0.0,1.0


Here, the row labels in the result are the distinct values occurring in all of the columns. The values are the respective counts of these values in each column.

There is also a `DataFrame.value_counts` method, but it computes counts considering each row of the DataFrame as a tuple to determine the number of occurrences of each distinct row:

In [199]:
data = pd.DataFrame({"a": [1, 1, 1, 2, 2], "b": [0, 0, 1, 0, 0]})

data

,a,b
0,1,0
1,1,0
2,1,1
3,2,0
4,2,0


In [200]:
data.value_counts()

a  b
1  0    2
2  0    2
1  1    1
Name: count, dtype: int64